# Database Source Data Preparation
Outputs 5 files into 'Resources/db_sources':
1. flights.zip - compressed version of 'flights.zip' with 2023 flight data from Bureau of Transportation Statistics (BTS)

2. airlines.csv - marketing and operating airline IATA codes mapped to the airline name

3. airports.csv - airports within the 50 U.S. States with domestic flight information

4. airport_sizes.csv - small, medium, large: 1, 2, 3 - reference for DB

5. states.csv - state abbreviations mapped to state name - reference for DB


In [1]:
# Import Dependencies
import pandas as pd
from pathlib import Path
import glob
import zipfile
import os
import shutil
import re

## Extract Raw Flight Data 
Downloaded from Bureau of BTS

In [2]:
# List of all source zips
zips = ['Resources/BTS/BTS_Q1_2023.zip','Resources/BTS/BTS_Q2_2023.zip',
       'Resources/BTS/BTS_Q3_2023.zip','Resources/BTS/BTS_Q4_2023.zip']

# Directory to extract to, confirm it exists
extraction_path = 'Resources/BTS/extracted_zips'
os.makedirs(extraction_path, exist_ok=True)

# Iterate through zips and extract all
for zip in zips:
    with zipfile.ZipFile(zip, 'r') as zip_ref:
        zip_ref.extractall(extraction_path)

    print(f"Extracted {zip} to {extraction_path}")
print('All files extracted.')

Extracted Resources/BTS/BTS_Q1_2023.zip to Resources/BTS/extracted_zips
Extracted Resources/BTS/BTS_Q2_2023.zip to Resources/BTS/extracted_zips
Extracted Resources/BTS/BTS_Q3_2023.zip to Resources/BTS/extracted_zips
Extracted Resources/BTS/BTS_Q4_2023.zip to Resources/BTS/extracted_zips
All files extracted.


In [3]:
# Read all extracted CSV's into a single dataframe
# glob is an efficient way to work through all CSV's in a directory: https://docs.python.org/3/library/glob.html
csv_files = glob.glob(f"{extraction_path}/*.csv")
# Empty list for storing DataFrames
dataframes = []
# counter for row sums
sum_rows = 0

# Iterate through csv's and store into DataFrames list
for csv_file in csv_files:
    df = pd.read_csv(csv_file, low_memory=False)
    dataframes.append(df)
    # count number of rows per dataframe to compare to final product
    row_count = len(df)
    sum_rows += row_count
    print(f"Number of rows in '{csv_file}': {row_count}")
# Concatenante into single DataFrame
unfiltered_flight_info = pd.concat(dataframes, ignore_index = True)
# Count total rows
total_rows = len(unfiltered_flight_info)
print("Total Rows:",total_rows)
print("Sum Rows:",sum_rows)
# Compare and print a message
if sum_rows == total_rows:
    print("The sum of all CSV rows matches the total rows in the concatenated DataFrame.")
else:
    print("There is a mismatch between the sum of all CSV rows and the total rows in the concatenated DataFrame.")

Number of rows in 'Resources/BTS/extracted_zips\On_Time_Marketing_Carrier_On_Time_Performance_2023_1.csv': 573877
Number of rows in 'Resources/BTS/extracted_zips\On_Time_Marketing_Carrier_On_Time_Performance_2023_10.csv': 635538
Number of rows in 'Resources/BTS/extracted_zips\On_Time_Marketing_Carrier_On_Time_Performance_2023_11.csv': 599814
Number of rows in 'Resources/BTS/extracted_zips\On_Time_Marketing_Carrier_On_Time_Performance_2023_12.csv': 606218
Number of rows in 'Resources/BTS/extracted_zips\On_Time_Marketing_Carrier_On_Time_Performance_2023_2.csv': 536229
Number of rows in 'Resources/BTS/extracted_zips\On_Time_Marketing_Carrier_On_Time_Performance_2023_3.csv': 616234
Number of rows in 'Resources/BTS/extracted_zips\On_Time_Marketing_Carrier_On_Time_Performance_2023_4.csv': 596676
Number of rows in 'Resources/BTS/extracted_zips\On_Time_Marketing_Carrier_On_Time_Performance_2023_5.csv': 616630
Number of rows in 'Resources/BTS/extracted_zips\On_Time_Marketing_Carrier_On_Time_Per

In [4]:
# display dataframe updating the options to see all headers
pd.set_option('display.max_columns', None)
unfiltered_flight_info.head(1)

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,Originally_Scheduled_Code_Share_Airline,DOT_ID_Originally_Scheduled_Code_Share_Airline,IATA_Code_Originally_Scheduled_Code_Share_Airline,Flight_Num_Originally_Scheduled_Code_Share_Airline,Operating_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,Flight_Number_Operating_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime,DivAirportLandings,DivReachedDest,DivActualElapsedTime,DivArrDelay,DivDistance,Div1Airport,Div1AirportID,Div1AirportSeqID,Div1WheelsOn,Div1TotalGTime,Div1LongestGTime,Div1WheelsOff,Div1TailNum,Div2Airport,Div2AirportID,Div2AirportSeqID,Div2WheelsOn,Div2TotalGTime,Div2LongestGTime,Div2WheelsOff,Div2TailNum,Div3Airport,Div3AirportID,Div3AirportSeqID,Div3WheelsOn,Div3TotalGTime,Div3LongestGTime,Div3WheelsOff,Div3TailNum,Div4Airport,Div4AirportID,Div4AirportSeqID,Div4WheelsOn,Div4TotalGTime,Div4LongestGTime,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Duplicate,Unnamed: 119
0,2023,1,1,22,7,2023-01-22,B6,B6,20409,B6,1447,NaN,NaN,NaN,NaN,B6,20409,B6,N636JB,1447,12197,1219702,31703,HPN,"White Plains, NY",NY,36,New York,22,15304,1530402,33195,TPA,"Tampa, FL",FL,12,Florida,33,800,756.0,-4.0,0.0,0.0,-1.0,0800-0859,12.0,808.0,1053.0,5.0,1102,1058.0,-4.0,0.0,0.0,-1.0,1100-1159,0.0,NaN,0.0,182.0,182.0,165.0,1.0,1032.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN


In [5]:
# Create new dataframe with desired columns
flights_df = pd.DataFrame(unfiltered_flight_info[['FlightDate','IATA_Code_Marketing_Airline',
                                                  'Operating_Airline ','Flight_Number_Marketing_Airline',
                                                  'Origin','Dest','DepDelay','DepDelayMinutes',
                                                  'DepDel15','ArrDelay','ArrDelayMinutes','Cancelled',
                                                  'Diverted','CarrierDelay','WeatherDelay','NASDelay',
                                                  'SecurityDelay','LateAircraftDelay']])
flights_df.head()

,FlightDate,IATA_Code_Marketing_Airline,Operating_Airline,Flight_Number_Marketing_Airline,Origin,Dest,DepDelay,DepDelayMinutes,DepDel15,ArrDelay,ArrDelayMinutes,Cancelled,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2023-01-22,B6,B6,1447,HPN,TPA,-4.0,0.0,0.0,-4.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,2023-01-22,B6,B6,1451,BOS,MCO,53.0,53.0,1.0,54.0,54.0,0.0,0.0,39.0,0.0,1.0,0.0,14.0
2,2023-01-22,B6,B6,1453,FLL,SJU,7.0,7.0,0.0,23.0,23.0,0.0,0.0,7.0,0.0,16.0,0.0,0.0
3,2023-01-22,B6,B6,1454,SJU,FLL,7.0,7.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,2023-01-22,B6,B6,1455,BOS,DCA,-8.0,0.0,0.0,-17.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [6]:
# Empty and delete the extraction_path directory to save space
try:
    shutil.rmtree(extraction_path)
    print("The DataFrame has the necessary information to proceed.")
    print(f"Directory {extraction_path} and all its contents have been successfully deleted.")
except OSError as e:
    print(f"Error: {e.strerror}")

The DataFrame has the necessary information to proceed.
Directory Resources/BTS/extracted_zips and all its contents have been successfully deleted.


## Initial Airport Selection

In [7]:
# Determine total number of unique airports in flight data
# Concatenate 'Origin' and 'Dest' columns to get all airports in one Series
all_airports = pd.concat([unfiltered_flight_info['Origin'], unfiltered_flight_info['Dest']])

# Calculate the number of unique airports
num_unique_airports = all_airports.nunique()

print("Total number of airports in flight data:", num_unique_airports)

Total number of airports in flight data: 359


In [8]:
#  compare us_airport data to airports in flight data
# Read in CSV of us-airports
us_airports = pd.read_csv('Resources/us-airports.csv')
# create exclusion
iata_match = us_airports['iata_code'].isin(flights_df['Origin'])
# filter the data
us_airports_filtered = us_airports[iata_match].reset_index(drop=True)
# count number of airports
print("Number of matching airports between data sets:", us_airports_filtered['iata_code'].nunique())

Number of matching airports between data sets: 351


In [9]:
# Which 8 airports do we have flight data for that our airport list doesn't have?
# create mask for inclusion
iata_match = flights_df['Origin'].isin(us_airports_filtered['iata_code'])
# drop data that doesn't match 
us_airports_missing = flights_df[~iata_match]
# show the extra airport codes in flight data
extra_airports = us_airports_missing['Origin'].unique()
print("Airports with flight data but no airport data:",extra_airports)

Airports with flight data but no airport data: ['SJU' 'STT' 'BQN' 'PSE' 'STX' 'PPG' 'GUM' 'SPN']


Researched IATA codes to determine location of airports(Google, https://www.iata.org/en/publications/directories/code-search/:
>SJU, BQM, PSE = Puerto Rico

>STT,STX, = U.S. Virgin Islands

>PPG = American Samoa (unincorporated territory)

>GUM = Guam

>SPN = Northern Mariana Islands

Conclusion: These 8 airports can be eliminated from flight data to include domestic flights and airports only within the 50 states.

Total airports: 351

## Preparing and Exporting Flight Data

In [10]:
# Excluding flight data from the 8 erroneous airports 
exclusions = (flights_df['Origin'].isin(extra_airports) | flights_df['Dest'].isin(extra_airports))

filtered_flights = flights_df[~exclusions]
print("Final number of airports in flight data:",filtered_flights['Origin'].nunique())

Final number of airports in flight data: 351


In [11]:
filtered_flights.head(1)

,FlightDate,IATA_Code_Marketing_Airline,Operating_Airline,Flight_Number_Marketing_Airline,Origin,Dest,DepDelay,DepDelayMinutes,DepDel15,ArrDelay,ArrDelayMinutes,Cancelled,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2023-01-22,B6,B6,1447,HPN,TPA,-4.0,0.0,0.0,-4.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [12]:
# standardizing column names, most other datasets are closer to the lowercase and _ separator standard
flights_final = filtered_flights.rename(columns={'FlightDate':'flight_date','IATA_Code_Marketing_Airline':'marketing_airline',
                                        'Operating_Airline ':'operating_airline',
                                        'Flight_Number_Marketing_Airline':'flight_number',
                                        'Origin':'origin_airport','DepDel15':'depart_15_min_delay',
                                        'Dest':'destination_airport','DepDelay':'departure_delay',
                                        'DepDelayMinutes':'depart_delay_minutes','ArrDelay':'arrival_delay',
                                        'ArrDelayMinutes':'arr_delay_minutes','Cancelled':'cancelled',
                                        'Diverted':'diverted','CarrierDelay':'carrier_delay',
                                        'WeatherDelay':'weather_delay','NASDelay':'nas_delay',
                                        'SecurityDelay':'security_delay','LateAircraftDelay':'late_aircraft_delay'})
flights_final.head()

,flight_date,marketing_airline,operating_airline,flight_number,origin_airport,destination_airport,departure_delay,depart_delay_minutes,depart_15_min_delay,arrival_delay,arr_delay_minutes,cancelled,diverted,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2023-01-22,B6,B6,1447,HPN,TPA,-4.0,0.0,0.0,-4.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,2023-01-22,B6,B6,1451,BOS,MCO,53.0,53.0,1.0,54.0,54.0,0.0,0.0,39.0,0.0,1.0,0.0,14.0
4,2023-01-22,B6,B6,1455,BOS,DCA,-8.0,0.0,0.0,-17.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
5,2023-01-22,B6,B6,1459,BDL,FLL,1.0,1.0,0.0,-1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
6,2023-01-22,B6,B6,1460,FLL,BDL,-8.0,0.0,0.0,-15.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [13]:
flights_final.dtypes

flight_date              object
marketing_airline        object
operating_airline        object
flight_number             int64
origin_airport           object
destination_airport      object
departure_delay         float64
depart_delay_minutes    float64
depart_15_min_delay     float64
arrival_delay           float64
arr_delay_minutes       float64
cancelled               float64
diverted                float64
carrier_delay           float64
weather_delay           float64
nas_delay               float64
security_delay          float64
late_aircraft_delay     float64
dtype: object

In [14]:
columns_with_float = ['departure_delay','depart_delay_minutes','depart_15_min_delay','arrival_delay',
                      'arr_delay_minutes','cancelled','diverted','carrier_delay','weather_delay',
                      'nas_delay','security_delay','late_aircraft_delay']

# Change dtype on each column with float that should have integers and NULL values
for column in columns_with_float:
    flights_final[column] = flights_final[column].astype('Int64')
print(flights_final.dtypes)

flight_date             object
marketing_airline       object
operating_airline       object
flight_number            int64
origin_airport          object
destination_airport     object
departure_delay          Int64
depart_delay_minutes     Int64
depart_15_min_delay      Int64
arrival_delay            Int64
arr_delay_minutes        Int64
cancelled                Int64
diverted                 Int64
carrier_delay            Int64
weather_delay            Int64
nas_delay                Int64
security_delay           Int64
late_aircraft_delay      Int64
dtype: object


In [15]:
# Export filtered data set to CSV
flights_csv_path = 'Resources/db_sources/flights.csv'
flights_final.to_csv(flights_csv_path,index=False)
# get the size of the csv
csv_file_size = os.path.getsize(flights_csv_path)
# convert to MB
csv_size_mb = csv_file_size / (1024 * 1024)
print(f"The full flight data CSV is {round(csv_size_mb,2)} MB.")

The full flight data CSV is 368.54 MB.


In [16]:
# Zip filtered_flights.csv and delete to allow github upload
zip_path = 'Resources/db_sources/flights.zip'

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(flights_csv_path, os.path.basename(flights_csv_path))

# Check if the ZIP file exists and the size condition is met
if os.path.exists(zip_path):
    # Get the size of the ZIP file in bytes
    zip_size = os.path.getsize(zip_path)
    zip_size_mb = zip_size / (1024 * 1024)
    # Define the size condition, e.g., file must be larger than 0 bytes to ensure it's not empty
    if zip_size_mb > 50:
        os.remove(flights_csv_path)
        print(f"The file {flights_csv_path} has been zipped to {round(zip_size_mb,2)} MB and the original CSV has been deleted.")
    else:
        print(f"The file {zip_path} is empty.")
else:
    print("Error: The zip file was not created successfully.")

The file Resources/db_sources/flights.csv has been zipped to 58.4 MB and the original CSV has been deleted.


## Airline Data

In [17]:
filtered_flights.columns

Index(['FlightDate', 'IATA_Code_Marketing_Airline', 'Operating_Airline ',
       'Flight_Number_Marketing_Airline', 'Origin', 'Dest', 'DepDelay',
       'DepDelayMinutes', 'DepDel15', 'ArrDelay', 'ArrDelayMinutes',
       'Cancelled', 'Diverted', 'CarrierDelay', 'WeatherDelay', 'NASDelay',
       'SecurityDelay', 'LateAircraftDelay'],
      dtype='object')

In [18]:
all_airlines = filtered_flights['Operating_Airline '].unique()
all_airlines

array(['B6', 'DL', 'YX', 'F9', 'G4', 'HA', 'MQ', 'NK', '9E', 'AA', 'PT',
       'YV', 'OH', 'OO', 'AS', 'QX', 'UA', 'C5', 'G7', 'ZW', 'WN'],
      dtype=object)

In [19]:
# Identify airlines and make list in matching order 
# Source: https://www.iata.org/en/publications/directories/code-search/
airline_dict = {'YV':'Mesa Airlines', 'OO':'SkyWest Airlines','QX':'Horizon Air Industries',
                 'G7':'GoJet Airlines','YX':'Republic Airways','C5':'CommuteAir','9E':'Endeavor Air',
                 'ZW':'Air Wisconsin Airlines','PT':'Piedmont Airlines','OH':'PSA Airlines',
                 'MQ':'Envoy Air','AA':'American Airlines','DL':'Delta Airlines','WN':'Southwest Airlines',
                 'UA':'United Airlines','AS':'Alaska Airlines', 'B6':'JetBlue','NK':'Spirit Airlines',
                 'F9':'Frontier Airlines','G4':'Allegiant Air','HA':'Hawaiian Airlines'}
# create dataframe for airlines
airlines_unsorted_df = pd.DataFrame(list(airline_dict.items()), columns=['iata_code', 'airline_name'])
airlines_df = airlines_unsorted_df.sort_values(by='airline_name').reset_index(drop=True)
airlines_df.head()

,iata_code,airline_name
0,ZW,Air Wisconsin Airlines
1,AS,Alaska Airlines
2,G4,Allegiant Air
3,AA,American Airlines
4,C5,CommuteAir


In [20]:
# Identify unique IATA codes and store in a list in order of frequency
marketing_airline_codes = filtered_flights['IATA_Code_Marketing_Airline'].value_counts().index.to_list()
marketing_airline_codes

['AA', 'DL', 'WN', 'UA', 'AS', 'B6', 'NK', 'F9', 'G4', 'HA']

In [21]:
# Identify codeshare airlines in order to compare to the major airline marketing partner
codeshare_airlines = list(set(all_airlines) - set(marketing_airline_codes))
codeshare_airlines

['YV', 'OO', 'ZW', 'C5', 'YX', 'G7', 'PT', 'MQ', '9E', 'QX', 'OH']

In [22]:
# In this data, do any of the codeshare airlines have more than one major airline marketing them?
unique_marketing_airlines = filtered_flights.groupby('Operating_Airline ')['IATA_Code_Marketing_Airline'].unique()
results = unique_marketing_airlines.apply(list).to_dict()
results
# Conclusion: Yes, some of the minor operating airlines are marketing by multiple major airlines.

{'9E': ['DL'],
 'AA': ['AA'],
 'AS': ['AS'],
 'B6': ['B6'],
 'C5': ['UA'],
 'DL': ['DL'],
 'F9': ['F9'],
 'G4': ['G4'],
 'G7': ['UA'],
 'HA': ['HA'],
 'MQ': ['AA'],
 'NK': ['NK'],
 'OH': ['AA'],
 'OO': ['AA', 'UA', 'AS', 'DL'],
 'PT': ['AA'],
 'QX': ['AS'],
 'UA': ['UA'],
 'WN': ['WN'],
 'YV': ['AA', 'UA'],
 'YX': ['DL', 'UA', 'AA'],
 'ZW': ['UA', 'AA']}

In [23]:
# Add a column to the airlines table identifying the airline as primary (marketing) or secondary (codesharing)
airlines_df['type'] = airlines_df['iata_code'].apply(lambda x: 'marketing' if x in marketing_airline_codes else ('codesharing' if x in codeshare_airlines else 'Unknown'))
airlines_df.head()

,iata_code,airline_name,type
0,ZW,Air Wisconsin Airlines,codesharing
1,AS,Alaska Airlines,marketing
2,G4,Allegiant Air,marketing
3,AA,American Airlines,marketing
4,C5,CommuteAir,codesharing


In [24]:
# Export to CSV
airlines_df.to_csv('Resources/db_sources/airlines.csv',index=False)
print("Airline data has been exported to csv.")

Airline data has been exported to csv.


## Airport Data

In [25]:
# Filter airports to just those that have flight data
airport_mask = us_airports['iata_code'].isin(filtered_flights['Origin'])
us_airports_filtered = pd.DataFrame(us_airports[airport_mask])
# confirm number of airports
print("Final number of airports in airport data:",us_airports_filtered['iata_code'].nunique())

Final number of airports in airport data: 351


In [26]:
us_airports_filtered.head(1)

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,country_name,iso_country,region_name,iso_region,local_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,score,last_updated
1,3632,KLAX,large_airport,Los Angeles International Airport,33.942501,-118.407997,125,NaN,United States,US,California,US-CA,CA,Los Angeles,1,KLAX,LAX,LAX,https://www.flylax.com/,https://en.wikipedia.org/wiki/Los_Angeles_Inte...,Tom Bradley,1335475,2023-12-21T12:31:02+00:00


#### Inputing City Data

In [27]:
# filter destination airports from flight data to unique entries
destinations_df = unfiltered_flight_info[['Dest', 'DestCityName']]
dest_filtered = destinations_df.drop_duplicates(subset=['Dest', 'DestCityName'])
dest_filtered.count()

Dest            359
DestCityName    359
dtype: int64

In [28]:
# check if there are any DestCityName rows that don't conform to City, State format
# regex = special characters include anything not a letter, number, comma, or whitespace
regex = r'^[A-Za-z0-9, ]+$'
# Dictionary to store results
not_uniform = []
# iterate through each row and check if cell contains comma AND doesn't match regex special char
for index, row in dest_filtered.iterrows():
    cell = row['DestCityName']
    if ',' in cell and not re.match(regex, cell):
        not_uniform.append(cell)

print(not_uniform)

# Learned: Even if there are slashes for metro areas, the format for ', state' still same

['West Palm Beach/Palm Beach, FL', 'Raleigh/Durham, NC', 'Sarasota/Bradenton, FL', 'Dallas/Fort Worth, TX', 'Montrose/Delta, CO', 'Bismarck/Mandan, ND', 'St. Louis, MO', 'Gulfport/Biloxi, MS', 'Jackson/Vicksburg, MS', 'Greensboro/High Point, NC', 'Cedar Rapids/Iowa City, IA', 'Jacksonville/Camp Lejeune, NC', 'Pasco/Kennewick/Richland, WA', 'Newburgh/Poughkeepsie, NY', 'Bloomington/Normal, IL', 'Allentown/Bethlehem/Easton, PA', 'St. Petersburg, FL', 'Bristol/Johnson City/Kingsport, TN', 'Elmira/Corning, NY', 'Mission/McAllen/Edinburg, TX', 'St. Cloud, MN', 'Clarksburg/Fairmont, WV', 'Manhattan/Ft. Riley, KS', 'Midland/Odessa, TX', 'Harlingen/San Benito, TX', 'Champaign/Urbana, IL', 'Scranton/Wilkes-Barre, PA', 'Charleston/Dunbar, WV', 'Ithaca/Cortland, NY', 'Saginaw/Bay City/Midland, MI', 'Newport News/Williamsburg, VA', 'New Bern/Morehead/Beaufort, NC', 'Lawton/Fort Sill, OK', 'Bend/Redmond, OR', 'Sun Valley/Hailey/Ketchum, ID', 'Beaumont/Port Arthur, TX', 'College Station/Bryan, TX', 

In [29]:
# Create airports DF with cities
# merge airports data with city data based on IATA code
airports_cities = us_airports_filtered.merge(dest_filtered, left_on='iata_code', 
                                            right_on='Dest', how ='inner')
# Get rid of the state abbreviation from cities column
airports_cities['city'] = airports_cities['DestCityName'].str.split(',').str[0]
#reset index
airports_cities.reset_index(drop=True).head()
# select columns
airports_cities.head()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,country_name,iso_country,region_name,iso_region,local_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,score,last_updated,Dest,DestCityName,city
0,3632,KLAX,large_airport,Los Angeles International Airport,33.942501,-118.407997,125,NaN,United States,US,California,US-CA,CA,Los Angeles,1,KLAX,LAX,LAX,https://www.flylax.com/,https://en.wikipedia.org/wiki/Los_Angeles_Inte...,Tom Bradley,1335475,2023-12-21T12:31:02+00:00,LAX,"Los Angeles, CA",Los Angeles
1,3754,KORD,large_airport,Chicago O'Hare International Airport,41.9786,-87.9048,680,NaN,United States,US,Illinois,US-IL,IL,Chicago,1,KORD,ORD,ORD,https://www.flychicago.com/ohare/home/pages/de...,https://en.wikipedia.org/wiki/O'Hare_Internati...,"CHI, Orchard Place",1503175,2024-03-09T23:28:49+00:00,ORD,"Chicago, IL",Chicago
2,3622,KJFK,large_airport,John F Kennedy International Airport,40.639447,-73.779317,13,NaN,United States,US,New York,US-NY,NY,New York,1,KJFK,JFK,JFK,https://www.jfkairport.com/,https://en.wikipedia.org/wiki/John_F._Kennedy_...,"Manhattan, New York City, NYC, Idlewild, IDL, ...",1052075,2022-10-18T18:49:55+00:00,JFK,"New York, NY",New York
3,3384,KATL,large_airport,Hartsfield Jackson Atlanta International Airport,33.6367,-84.428101,1026,NaN,United States,US,Georgia,US-GA,GA,Atlanta,1,KATL,ATL,ATL,http://www.atlanta-airport.com/,https://en.wikipedia.org/wiki/Hartsfield–Jacks...,NaN,2002475,2018-09-19T14:50:01+00:00,ATL,"Atlanta, GA",Atlanta
4,3878,KSFO,large_airport,San Francisco International Airport,37.61899948120117,-122.375,13,NaN,United States,US,California,US-CA,CA,San Francisco,1,KSFO,SFO,SFO,http://www.flysfo.com/,https://en.wikipedia.org/wiki/San_Francisco_In...,"QSF, QBA",1112475,2008-06-13T14:30:04+00:00,SFO,"San Francisco, CA",San Francisco


#### Updating Airport Types

In [30]:
# Checking airport types
airports_cities['type'].value_counts()

type
medium_airport    276
large_airport      65
small_airport      10
Name: count, dtype: int64

In [31]:
# rather than storing strings in the database, let's store size as an integer
airports_updated = airports_cities.copy()
# Get rid of _airport from 'type' column 
airports_updated['airport_size'] = airports_updated['type'].str.split('_').str[0]

size_mapping = {'small': 1, 'medium': 2, 'large': 3}
airports_updated['airport_size'] = airports_updated['airport_size'].replace(size_mapping)
# show new column created
airports_updated.head(1)

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,country_name,iso_country,region_name,iso_region,local_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,score,last_updated,Dest,DestCityName,city,airport_size
0,3632,KLAX,large_airport,Los Angeles International Airport,33.942501,-118.407997,125,NaN,United States,US,California,US-CA,CA,Los Angeles,1,KLAX,LAX,LAX,https://www.flylax.com/,https://en.wikipedia.org/wiki/Los_Angeles_Inte...,Tom Bradley,1335475,2023-12-21T12:31:02+00:00,LAX,"Los Angeles, CA",Los Angeles,3


#### Clean Up and Export

In [32]:
# drop and rename columns
airports_drop = airports_updated.drop(columns=['Dest','DestCityName','type'])
airports_df = airports_drop.rename(columns={'name':'airport_name',
                                              'local_region':'state_abbrv'})
airports_df.head()

,id,ident,airport_name,latitude_deg,longitude_deg,elevation_ft,continent,country_name,iso_country,region_name,iso_region,state_abbrv,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,score,last_updated,city,airport_size
0,3632,KLAX,Los Angeles International Airport,33.942501,-118.407997,125,NaN,United States,US,California,US-CA,CA,Los Angeles,1,KLAX,LAX,LAX,https://www.flylax.com/,https://en.wikipedia.org/wiki/Los_Angeles_Inte...,Tom Bradley,1335475,2023-12-21T12:31:02+00:00,Los Angeles,3
1,3754,KORD,Chicago O'Hare International Airport,41.9786,-87.9048,680,NaN,United States,US,Illinois,US-IL,IL,Chicago,1,KORD,ORD,ORD,https://www.flychicago.com/ohare/home/pages/de...,https://en.wikipedia.org/wiki/O'Hare_Internati...,"CHI, Orchard Place",1503175,2024-03-09T23:28:49+00:00,Chicago,3
2,3622,KJFK,John F Kennedy International Airport,40.639447,-73.779317,13,NaN,United States,US,New York,US-NY,NY,New York,1,KJFK,JFK,JFK,https://www.jfkairport.com/,https://en.wikipedia.org/wiki/John_F._Kennedy_...,"Manhattan, New York City, NYC, Idlewild, IDL, ...",1052075,2022-10-18T18:49:55+00:00,New York,3
3,3384,KATL,Hartsfield Jackson Atlanta International Airport,33.6367,-84.428101,1026,NaN,United States,US,Georgia,US-GA,GA,Atlanta,1,KATL,ATL,ATL,http://www.atlanta-airport.com/,https://en.wikipedia.org/wiki/Hartsfield–Jacks...,NaN,2002475,2018-09-19T14:50:01+00:00,Atlanta,3
4,3878,KSFO,San Francisco International Airport,37.61899948120117,-122.375,13,NaN,United States,US,California,US-CA,CA,San Francisco,1,KSFO,SFO,SFO,http://www.flysfo.com/,https://en.wikipedia.org/wiki/San_Francisco_In...,"QSF, QBA",1112475,2008-06-13T14:30:04+00:00,San Francisco,3


In [33]:
# Select columns and create new dataframe
airports_columns_df = pd.DataFrame(airports_updated[['iata_code', 'airport_size', 'name', 'latitude_deg', 
                                                   'longitude_deg','city','local_region']])
# rename columns
airports_final_df = airports_columns_df.rename(columns={'name':'airport_name',
                                              'local_region':'state_abbrev'})
# diplay final df for airports
airports_final_df.head()

,iata_code,airport_size,airport_name,latitude_deg,longitude_deg,city,state_abbrev
0,LAX,3,Los Angeles International Airport,33.942501,-118.407997,Los Angeles,CA
1,ORD,3,Chicago O'Hare International Airport,41.9786,-87.9048,Chicago,IL
2,JFK,3,John F Kennedy International Airport,40.639447,-73.779317,New York,NY
3,ATL,3,Hartsfield Jackson Atlanta International Airport,33.6367,-84.428101,Atlanta,GA
4,SFO,3,San Francisco International Airport,37.61899948120117,-122.375,San Francisco,CA


In [34]:
# export final airport data
airports_final_df.to_csv('Resources/db_sources/airports.csv',index=False)
print("Exported airport data to csv")

Exported airport data to csv


In [35]:
# create airport sizes reference
airport_sizes = pd.DataFrame({'size_id':[1,2,3],
                             'relative_size':['small','medium','large']})
airport_sizes

,size_id,relative_size
0,1,small
1,2,medium
2,3,large


In [36]:
# export size reference
airport_sizes.to_csv('Resources/db_sources/airport_sizes.csv', index=False)
print("Exported airport size reference to csv")

Exported airport size reference to csv


## States Data

In [37]:
# creating dataframe for the states 

# create lists of states and their abbreviations
state_abbrevs = list(airports_cities['local_region'].unique())
states = list(airports_cities['region_name'].unique())
# print lists to show they are in the same order
print(state_abbrevs)
print(states)
#create dataframe from the lists of states and their abbreviations
states_df = pd.DataFrame({'state_abbrev':state_abbrevs,
                         'state':states})
# alphabetical order by state name
states_sorted = states_df.sort_values(by='state').reset_index(drop=True)

# display 
states_sorted.head()

['CA', 'IL', 'NY', 'GA', 'NJ', 'TX', 'NV', 'FL', 'CO', 'VA', 'AZ', 'WA', 'MA', 'PA', 'NC', 'DC', 'MI', 'MN', 'UT', 'MD', 'MO', 'HI', 'OR', 'LA', 'TN', 'KY', 'OH', 'IN', 'NM', 'WI', 'CT', 'AK', 'OK', 'NE', 'RI', 'SC', 'AL', 'ID', 'KS', 'ME', 'IA', 'AR', 'NH', 'VT', 'SD', 'WY', 'MT', 'ND', 'MS', 'WV']
['California', 'Illinois', 'New York', 'Georgia', 'New Jersey', 'Texas', 'Nevada', 'Florida', 'Colorado', 'Virginia', 'Arizona', 'Washington', 'Massachusetts', 'Pennsylvania', 'North Carolina', 'District of Columbia', 'Michigan', 'Minnesota', 'Utah', 'Maryland', 'Missouri', 'Hawaii', 'Oregon', 'Louisiana', 'Tennessee', 'Kentucky', 'Ohio', 'Indiana', 'New Mexico', 'Wisconsin', 'Connecticut', 'Alaska', 'Oklahoma', 'Nebraska', 'Rhode Island', 'South Carolina', 'Alabama', 'Idaho', 'Kansas', 'Maine', 'Iowa', 'Arkansas', 'New Hampshire', 'Vermont', 'South Dakota', 'Wyoming', 'Montana', 'North Dakota', 'Mississippi', 'West Virginia']


,state_abbrev,state
0,AL,Alabama
1,AK,Alaska
2,AZ,Arizona
3,AR,Arkansas
4,CA,California


In [38]:
# export states df to csv
states_sorted.to_csv('Resources/db_sources/states.csv',index=False)
print('Exported State information')

Exported State information
